In [ ]:
!pip install -q unsloth
# Also get the latest nightly Unsloth!
!pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 133.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Unsloth also supports RoPE (Rotary Positinal Embedding) scaling internally.
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # Will load the 4Bit Quantized Model
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # a higher alpha value assigns more weight to the LoRA activations
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.12.9 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/data_spam_project

import pandas as pd
train_spam_data = pd.read_csv('train_spam_data.csv')
test_spam_data = pd.read_csv('test_spam_data.csv')


Mounted at /content/drive
/content/drive/MyDrive/data_spam_project


In [ ]:
from datasets import Dataset
def format_prompt(row):
    return f"""### Instruction:
Classify the following SMS message as SPAM or HAM.

### Message:
{row['text']}

### Response:
{row['label'].upper()}"""

train_spam_data["formatted_text"] = train_spam_data.apply(format_prompt, axis=1)

dataset = Dataset.from_pandas(train_spam_data[["formatted_text"]])


def tokenize_fn(examples):
    return tokenizer(
        examples["formatted_text"],
        truncation=True,
        max_length=2048,
        padding=False,
    )

tokenized_dataset = dataset.map(tokenize_fn, batched=True)


Map:   0%|          | 0/5397 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=1000,          # You can tune this
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        output_dir="./spam_llama",
        save_strategy="steps",
        save_steps=200,
        report_to="none",
    ),
)
trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,397 | Num Epochs = 6 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,4.409700
20,3.834400
30,2.998800
40,2.648900
50,2.619700
60,2.446300
70,2.371700
80,2.270000
90,2.309900
100,2.131500


Step,Training Loss
10,4.409700
20,3.834400
30,2.998800
40,2.648900
50,2.619700
60,2.446300
70,2.371700
80,2.270000
90,2.309900
100,2.131500


TrainOutput(global_step=1000, training_loss=1.5789566764831542, metrics={'train_runtime': 1463.952, 'train_samples_per_second': 21.859, 'train_steps_per_second': 0.683, 'total_flos': 1.4207731365003264e+16, 'train_loss': 1.5789566764831542, 'epoch': 5.9185185185185185})

In [ ]:
save_dir = "./spam_llm_finetuned"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./spam_llm_finetuned",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)



prompt = """### Instruction:
Classify the following SMS message as SPAM or HAM.

### Message:
Email AlertFrom: Jeri StewartSize: 2KBSubject: Low-cost prescripiton drvgsTo listen to email call 123
#### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=5)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
### Instruction:
Classify the following SMS message as SPAM or HAM.

### Message:
Email AlertFrom: Jeri StewartSize: 2KBSubject: Low-cost prescripiton drvgsTo listen to email call 123
#### Response:
SPAM

### Response


In [23]:
prompt = """### Instruction:
Classify the following SMS message as SPAM or HAM.

### Message:

Send a logo 2 ur lover - 2 names joined by a heart. Txt LOVE NAME1 NAME2 MOBNO eg LOVE ADAM EVE 07123456789 to 87077 Yahoo! POBox36504W45WQ TxtNO 4 no ads 150p
#### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=5)

out = tokenizer.decode(outputs[0], skip_special_tokens=True)
out, out.strip().split()[-3]

("### Instruction:\nClassify the following SMS message as SPAM or HAM.\n\n### Message:\n\nErr... Cud do. I'm going to  at 8pm. I haven't got a way to contact him until then.#### Response:\nHAM\n\n### Response:\n",
 'HAM')

In [24]:

df = test_spam_data
def predict(text):
    prompt = f"""### Instruction:
Classify the following SMS message as SPAM or HAM.

### Message:
{text}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=5)
    out = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return out.strip().split()[-3]  # SPAM or HAM

df["prediction"] = df["text"].apply(predict)
df.to_csv("test_with_predictions2.csv", index=False)


HAM
HAM
HAM
HAM
HAM
HAM
HAM
SPAM
SPAM
HAM
HAM
HAM
SPAM
SPAM
SPAM
U
HAM
HAM
SPAM
SPAM
HAM
SPAM
HAM.
HAM
HAM
Enjoy
SPAM
HAM
SPAM
SPAM
HAM
HAM
SPAM
SPAM
HAM
SPAM
SPAM
Good
SPAM
SPAM
SPAM
HAM
SPAM
HAM
HAM
SPAM
SPAM
SPAM
HAM
HAM
HAM
SPAM
HAM
HAM
HAM
HAM
SPAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
SPAM
HAM
SPAM
SPAM
HAM
HAM
HAM
SPAM
HAM
SPAM
SPAM
SPAM
HAM
HAM
HAM
HAM
SPAM
SPAM
HAM
HAM
HAM
SPAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
Response:
HAM.
You
HAM
HAM
HAM
HAM
HAM
HAM
HAM
Enjoy
SPAM
HAM
HAM
HAM.
SPAM
HAM
SPAM
HAM
HAM
SPAM
HAM
HAM
You
SPAM
HAM
HAM
HAM
HAM
SPAM
SPAM
HAM
HAM
HAM
HAM
Response:
HAM
HAM
HAM
SPAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
Enjoy
HAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
HAM
SPAM
HAM
SPAM
HAM
HAM.
HAM
HAM
HAM
HAM
HAM
SPAM
HAM
HAM
SPAM
Have
HAM
SPAM
HAM
HAM
HAM


In [25]:
import pandas as pd


# Normalize labels
df["label_norm"] = df["label"].str.strip().str.upper()
df["pred_norm"] = df["prediction"].str.strip().str.upper()

# Correct / incorrect
df["correct"] = df["label_norm"] == df["pred_norm"]

# Metrics
total = len(df)
correct = df["correct"].sum()
accuracy = correct / total

print(f"Total samples: {total}")
print(f"Correct predictions: {correct}")
print(f"Accuracy: {accuracy:.4f}")


Total samples: 175
Correct predictions: 160
Accuracy: 0.9143


In [26]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(df["label_norm"], df["pred_norm"]))
print(classification_report(df["label_norm"], df["pred_norm"]))


[[  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  3   1 113   4   1   0   0   1   2]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   1   0   0   2  47   0   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0]]
              precision    recall  f1-score   support

       ENJOY       0.00      0.00      0.00         0
        GOOD       0.00      0.00      0.00         0
         HAM       0.99      0.90      0.95       125
        HAM.       0.00      0.00      0.00         0
        HAVE       0.00      0.00      0.00         0
   RESPONSE:       0.00      0.00      0.00         0
        SPAM       1.00      0.94      0.97        50
           U       0.00      0.00      0.00         0
         YOU       0.00      0.00      0.00         0

    accuracy                           0.91       175
   macro avg       0.22      0.20      0.21       17

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
